In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
df =pd.read_excel("df_limpio.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   folio               22735 non-null  int64         
 1   tag                 22735 non-null  object        
 2   folio_solicitud     22735 non-null  int64         
 3   fecha               22735 non-null  datetime64[ns]
 4   marca               22735 non-null  object        
 5   modelo              22735 non-null  object        
 6   plazo               22735 non-null  object        
 7   precio              22735 non-null  float64       
 8   enganche            22735 non-null  float64       
 9   descuento           22735 non-null  float64       
 10  semana              22735 non-null  int64         
 11  monto_financiado    22735 non-null  float64       
 12  costo_total         22735 non-null  int64         
 13  monto_accesorios    22735 non-null  float64   

**PRECIO**

Vamos a analizar la variable "precio" debido a que esta será nuestra varaible dependinte que pasará a convertirse en variable diatónica.
Gama baja - Gama media

In [3]:
precioStats = df["precio"].describe()
print(precioStats)

count    22735.000000
mean      3997.036591
std       1361.507164
min        973.240000
25%       3049.000000
50%       3549.000000
75%       4899.000000
max      14900.000000
Name: precio, dtype: float64


In [4]:
# Elegimos las variables de interés para poner a prueba el modelo
precio = df.iloc[ : , [8,9,11,12,13,25,26,27,29,30,7]]

In [5]:

# Calcular la media de la columna
meanPrecio = precio["precio"].mean()

# Definir una función que asignará la categoría
def categorizar_valor(valor):
    if valor > meanPrecio:
        return 'Gama media'
    else:
        return 'Gama baja'

# Aplicar la función a la columna y crear una nueva columna con las categorías
precio['PrecioGama'] = precio["precio"].apply(categorizar_valor)

# Mostrar el DataFrame resultante
print(df)


       folio       tag  folio_solicitud               fecha     marca  \
0          3  HDNAWQ3A               30 2021-11-12 19:00:00       ZTE   
1          4  M418YNR4               33 2021-11-12 19:15:00  MOTOROLA   
2          5  SZ7V3NZT               37 2021-11-13 14:13:00       ZTE   
3          6  9I08UIQZ               38 2021-11-13 15:45:00       ZTE   
4          7  JMF4LKYU               44 2021-11-13 16:10:00  MOTOROLA   
...      ...       ...              ...                 ...       ...   
22730  22967  92KQVRIX            96502 2023-04-18 18:45:00  Motorola   
22731  22968  NWB2EA8J            96479 2023-04-18 18:46:00  Motorola   
22732  22969  VDTRJ2W6            96442 2023-04-18 18:47:00   Samsung   
22733  22970  PETLF56C            96536 2023-04-18 18:49:00   Samsung   
22734  22971  2CI4N2WS            96534 2023-04-18 18:49:00   Samsung   

                             modelo plazo  precio  enganche  descuento  ...  \
0                Blade A3 2020 RO B   26S  1

C:\Users\Carlos Casillas\AppData\Local\Temp\ipykernel_15964\2003049238.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precio['PrecioGama'] = precio["precio"].apply(categorizar_valor)


**PUNTOS**

**SOCREBURO**

**PORCENTAJE DE ENGANCHE**